In [3]:
import tensorflow as tf
import os
from subprocess import call
import pandas as pd
import imageio
import scipy.misc
import skimage
from tensorflow.keras import layers

In [4]:
df = pd.read_csv('HAM10000_metadata.csv')
df

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [5]:
output_shape = (150,200)
df["image"] = df["image_id"].map((lambda id : skimage.transform.resize(imageio.imread("train/" + id + ".jpg"), output_shape)))

/Users/dylancordaro/miniconda3/envs/tensorflow_env/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/dylancordaro/miniconda3/envs/tensorflow_env/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [6]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#plt.imshow(df.loc[9,"image"]);plt.show()

In [7]:
#plt.imshow(imageio.imread("train/ISIC_0025209.jpg"));plt.show()

In [8]:
labels = {"bkl" : 0, "nv": 1, "df": 2, "mel":3, "vasc":4, "bcc":5, "akiec":6}
images=df["image"]
diagnosis=df.dx.unique()
target = list(enumerate(diagnosis))


In [9]:
targets = df["dx"].map(lambda x : labels.get(x))

In [10]:
from sklearn.model_selection import train_test_split
test_size = .15
val_size = .15
X_train = []
X_test = []
y_train = []
y_test = []
X_val = []
y_val = []
for i in range(7):
    indices = (targets == i)
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(images[indices], targets[indices], test_size = test_size)
    #print(X_train_temp)
    X_train_temp, X_val_temp, y_train_temp, y_val_temp = train_test_split(X_train_temp, y_train_temp, test_size = val_size/(1-test_size))
    X_train = X_train + list(X_train_temp)
    X_test = X_test + list(X_test_temp)
    y_train = y_train + list(y_train_temp)
    y_test = y_test + list(y_test_temp)
    X_val = X_val + list(X_val_temp )
    y_val = y_val + list(y_val_temp)

In [11]:
#images.shape
#images[targets == 5]
#[] + list(X_train_temp)

In [12]:
input_shape = (150, 200, 3)
num_classes = 7

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(layers.Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(layers.MaxPool2D(pool_size = (2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(layers.MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.40))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 200, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 200, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 100, 32)       0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 100, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 100, 64)       18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 100, 64)       36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 50, 64)        0         
__________

In [13]:
optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [14]:
import numpy as np
y_train_real = np.zeros((len(y_train), 7))
for i in range(len(y_train)):
    y_train_real[i,y_train[i]] = 1
    
y_val_real = np.zeros((len(y_val), 7))
for i in range(len(y_val)):
    y_val_real[i,y_val[i]] = 1
X_train = np.array(X_train)
X_val = np.array(X_val)

In [21]:
# # Fit the model
epochs = 50 
batch_size = 10
model.fit(X_train, np.array(y_train_real), epochs=10, batch_size=32, callbacks=[learning_rate_reduction], validation_data = (X_val, y_val_real))

Train on 7005 samples, validate on 1504 samples
Epoch 1/10
7005/7005 [==============================] - 1737s 248ms/step - loss: 1.1063 - acc: 0.6664 - val_loss: 0.9710 - val_acc: 0.6689
Epoch 2/10
7005/7005 [==============================] - 1453s 207ms/step - loss: 0.9559 - acc: 0.6712 - val_loss: 0.9041 - val_acc: 0.6695
Epoch 3/10
7005/7005 [==============================] - 1175s 168ms/step - loss: 0.8979 - acc: 0.6718 - val_loss: 0.8505 - val_acc: 0.6715
Epoch 4/10
7005/7005 [==============================] - 1187s 170ms/step - loss: 0.8886 - acc: 0.6795 - val_loss: 0.8667 - val_acc: 0.6742
Epoch 5/10
7005/7005 [==============================] - 1204s 172ms/step - loss: 0.8559 - acc: 0.6819 - val_loss: 0.8973 - val_acc: 0.6835
Epoch 6/10
7005/7005 [==============================] - 1172s 167ms/step - loss: 0.8413 - acc: 0.6855 - val_loss: 0.8068 - val_acc: 0.6862
Epoch 7/10
7005/7005 [==============================] - 1165s 166ms/step - loss: 0.8165 - acc: 0.6935 - val_loss: 0.79

In [19]:
print(X_val.shape,y_val_real.shape)

(1504, 150, 200, 3) (1504, 7)
